In [1]:
# # ===========================================
# #  LungVAE (PyTorch, state_dict) => Predict 3 NORMAL & 3 PNEUMONIA
# #  Dùng class uVAE mà bạn đã cung cấp
# # ===========================================
# import os, cv2, glob, numpy as np
# import torch
# import torch.nn.functional as F
# import matplotlib.pyplot as plt

# # ==== (1) Dán class uVAE và convBlock bạn gửi vào đây (không đổi gì) ====
# import torch
# import torch.nn as nn
# from torch.distributions import Normal, Independent
# from torch.distributions.kl import kl_divergence as KLD
# import numpy as np
# from torch.nn.functional import softplus, sigmoid, softmax
# import pdb
# import torch.nn.functional as F
# _device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# class convBlock(nn.Module):
#     def __init__(self, inCh, nhid, nOp, pool=True,
#                     ker=3,padding=1,pooling=2):
#         super(convBlock,self).__init__()

#         self.enc1 = nn.Conv2d(inCh,nhid,kernel_size=ker,padding=1)
#         self.enc2 = nn.Conv2d(nhid,nOp,kernel_size=ker,padding=1)
#         self.bn = nn.BatchNorm2d(inCh)    

#         if pool:
#             self.scale = nn.AvgPool2d(kernel_size=pooling)
#         else:
#             self.scale = nn.Upsample(scale_factor=pooling)
#         self.pool = pool
#         self.act = nn.ReLU()

#     def forward(self,x):
#         x = self.scale(x)
#         x = self.bn(x)
#         x = self.act(self.enc1(x))
#         x = self.act(self.enc2(x))
#         return x

# class uVAE(nn.Module):
#     def __init__(self, nlatent,unet=False, 
#                     nhid=8, ker=3, inCh=1,h=640,w=512):
#         super(uVAE, self).__init__()
#         self.latent_space = nlatent
#         self.unet = unet

#         if not self.unet:
#             # VAE Encoder
#             self.enc11 = nn.Conv2d(inCh,nhid,kernel_size=ker,padding=1)
#             self.enc12 = nn.Conv2d(nhid,nhid,kernel_size=ker,padding=1)

#             self.enc2 = convBlock(nhid,2*nhid,2*nhid,pool=True)
#             self.enc3 = convBlock(2*nhid,4*nhid,4*nhid,pool=True)
#             self.enc4 = convBlock(4*nhid,8*nhid,8*nhid,pool=True)
#             self.enc5 = convBlock(8*nhid,16*nhid,16*nhid,pool=True)

#             self.bot11 = nn.Conv1d(16*nhid,1,kernel_size=1)
#             self.bot12 = nn.Conv1d(int((h/16)*(w/16)),2*nlatent,kernel_size=1)

#             # Decoder (VAE path)
#             self.bot21 = nn.Conv1d(nlatent,int((h/64)*(w/64)),kernel_size=1)
#             self.bot22 = nn.Conv1d(1,nhid,kernel_size=1)
#             self.bot23 = nn.Conv1d(nhid,4*nhid,kernel_size=1)
#             self.bot24 = nn.Conv1d(4*nhid,16*nhid,kernel_size=1)

#         # U-Net Encoder
#         self.uEnc11 = nn.Conv2d(inCh,nhid,kernel_size=ker,padding=1)
#         self.uEnc12 = nn.Conv2d(nhid,nhid,kernel_size=ker,padding=1)

#         self.uEnc2 = convBlock(nhid,2*nhid,2*nhid,pool=True,pooling=4)
#         self.uEnc3 = convBlock(2*nhid,4*nhid,4*nhid,pool=True,pooling=4)
#         self.uEnc4 = convBlock(4*nhid,8*nhid,8*nhid,pool=True)
#         self.uEnc5 = convBlock(8*nhid,16*nhid,16*nhid,pool=True)

#         # Joint decoder
#         if not self.unet:
#             self.dec5 = convBlock(32*nhid,8*nhid,8*nhid,pool=False)
#         else:
#             self.dec5 = convBlock(16*nhid,8*nhid,8*nhid,pool=False)

#         self.dec4 = convBlock(16*nhid,4*nhid,4*nhid,pool=False)
#         self.dec3 = convBlock(8*nhid,2*nhid,2*nhid,pool=False,pooling=4)
#         self.dec2 = convBlock(4*nhid,nhid,nhid,pool=False,pooling=4)

#         self.dec11 = nn.Conv2d(2*nhid,nhid,kernel_size=ker,padding=1)
#         self.dec12 = nn.Conv2d(nhid,inCh,kernel_size=ker,padding=1)
        
#         self.act = nn.ReLU()
#         self.mu_0 = torch.zeros((1,nlatent)).to(_device)
#         self.sigma_0 = torch.ones((1,nlatent)).to(_device)

#         self.h = h
#         self.w = w

#     def vae_encoder(self,x):
#         x = self.act(self.enc11(x))
#         x = self.act(self.enc12(x))
#         x = self.enc2(x)
#         x = self.enc3(x)
#         x = self.enc4(x)
#         x = self.enc5(x)

#         z = self.act(self.bot11(x.view(x.shape[0],x.shape[1],-1)))
#         z = self.bot12(z.permute(0,2,1))
#         return z.squeeze(-1)

#     def unet_encoder(self,x_in):
#         x = []
#         x.append(self.act(self.uEnc12(self.act(self.uEnc11(x_in)))))
#         x.append(self.uEnc2(x[-1]))
#         x.append(self.uEnc3(x[-1]))
#         x.append(self.uEnc4(x[-1]))
#         x.append(self.uEnc5(x[-1]))
#         return x

#     def decoder(self,x_enc,z=None):
#         if not self.unet:
#             x = self.act(self.bot21(z.unsqueeze(2)))
#             x = self.act(self.bot22(x.permute(0,2,1)))
#             x = self.act(self.bot23(x))
#             x = self.act(self.bot24(x))
#             x = x.view(x.shape[0],x.shape[1],int(self.h/64),int(self.w/64))
#             x = torch.cat((x,x_enc[-1]),dim=1)
#             x = self.dec5(x)
#         else:
#             x = self.dec5(x_enc[-1])
#         x = torch.cat((x,x_enc[-2]),dim=1)
#         x = self.dec4(x)
#         x = torch.cat((x,x_enc[-3]),dim=1)
#         x = self.dec3(x)
#         x = torch.cat((x,x_enc[-4]),dim=1)
#         x = self.dec2(x)
#         x = torch.cat((x,x_enc[-5]),dim=1)
#         x = self.act(self.dec11(x))
#         x = self.dec12(x)
#         return x

#     def forward(self, x):
#         kl = torch.zeros(1).to(_device)
#         z = 0.
#         x_enc = self.unet_encoder(x)
#         if not self.unet:
#             emb = self.vae_encoder(x)
#             mu, log_var = torch.chunk(emb, 2, dim=1)
#             log_var = softplus(log_var)
#             sigma = torch.exp(log_var / 2)
#             posterior = Independent(Normal(loc=mu,scale=sigma),1)
#             z = posterior.rsample()
#             prior = Independent(Normal(loc=self.mu_0,scale=self.sigma_0),1)
#             kl = KLD(posterior,prior).sum()
#         xHat = self.decoder(x_enc,z)
#         return kl, xHat
# # ==== (hết phần class) ===================================

# # ----------------------------
# # 2) Đường dẫn & cấu hình
# # ----------------------------
# WEIGHTS_PATH = "/kaggle/input/lungvae/lungVAE.pt"
# NORMAL_DIR = "/kaggle/input/chest-xray-pneumonia/chest_xray/chest_xray/test/NORMAL"
# PNEU_DIR   = "/kaggle/input/chest-xray-pneumonia/chest_xray/chest_xray/test/PNEUMONIA"
# SAVE_DIR = "./masks_demo_lungvae"
# os.makedirs(SAVE_DIR, exist_ok=True)

# DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
# print("[INFO] Device:", DEVICE)

# # ----------------------------
# # 3) Suy luận hyperparam từ state_dict & khởi tạo model
# # ----------------------------
# def infer_hparams_from_state_dict(sd):
#     """
#     Trích nlatent, nhid, h, w, unet từ state_dict.
#     - nlatent: in_channels của bot21.weight (nếu tồn tại) -> VAE on (unet=False)
#     - nhid: out_channels của uEnc11.weight
#     - (h/16)*(w/16): in_channels của bot12.weight (nếu có)
#     - Nếu không có bot21/bot12 -> khả năng model train ở chế độ unet=True (không VAE)
#     """
#     nhid = sd["uEnc11.weight"].shape[0] if "uEnc11.weight" in sd else 8

#     # Mặc định đoán unet=False nếu thấy các layer VAE
#     has_vae = ("bot21.weight" in sd) or ("bot12.weight" in sd)
#     unet_flag = not has_vae  # nếu không có VAE layers -> unet=True

#     # nlatent
#     nlatent = 16
#     if "bot21.weight" in sd:
#         # shape: [out_c, in_c, 1] -> in_c = nlatent
#         nlatent = sd["bot21.weight"].shape[1]

#     # h,w từ bot12.weight (Conv1d in_channels = (h/16)*(w/16))
#     h, w = 640, 512
#     if "bot12.weight" in sd:
#         in_c = sd["bot12.weight"].shape[1]
#         # Tìm (h,w) chia hết cho 16, sao cho (h/16)*(w/16) = in_c
#         candidates = []
#         for H in [512, 576, 600, 640, 672, 704, 720, 736, 768, 800, 832, 896, 960, 1024]:
#             for W in [384, 400, 448, 480, 512, 544, 576, 600, 640, 672, 704, 720, 736, 768, 800, 896, 960, 1024]:
#                 if (H % 16 == 0) and (W % 16 == 0):
#                     if (H//16)*(W//16) == in_c:
#                         candidates.append((H,W))
#         if candidates:
#             # ưu tiên (640,512) nếu có
#             if (640,512) in candidates:
#                 h, w = 640, 512
#             else:
#                 h, w = candidates[0]
#     return nlatent, nhid, h, w, unet_flag

# # Tải state_dict
# obj = torch.load(WEIGHTS_PATH, map_location="cpu")
# if "state_dict" in obj:
#     sd = obj["state_dict"]
# else:
#     # Có thể là state_dict thuần
#     if isinstance(obj, dict):
#         sd = obj
#     else:
#         raise RuntimeError("File weights không phải state_dict. (Nếu là full model, bạn có thể load trực tiếp.)")

# # Khởi tạo model theo state_dict
# nlatent, nhid, Himg, Wimg, unet_flag = infer_hparams_from_state_dict(sd)
# print(f"[INFO] Inferred: nlatent={nlatent}, nhid={nhid}, H×W={Himg}×{Wimg}, unet={unet_flag}")

# model = uVAE(nlatent=nlatent, unet=unet_flag, nhid=nhid, inCh=1, h=Himg, w=Wimg)
# missing, unexpected = model.load_state_dict(sd, strict=False)
# print("[INFO] load_state_dict: missing =", missing[:6], ("...(+%d)"%(max(0,len(missing)-6))) if len(missing)>6 else "")
# print("[INFO] load_state_dict: unexpected =", unexpected)
# model.to(DEVICE).eval()

# # ----------------------------
# # 4) Helpers (giữ API giống phiên bản TF/Keras)
# # ----------------------------
# def is_image_file(p): 
#     return p.lower().endswith((".png",".jpg",".jpeg",".bmp",".tif",".tiff"))

# def pick_samples(folder, k=3, seed=2025):
#     rng = np.random.default_rng(seed)
#     files = sorted([os.path.join(folder,f) for f in os.listdir(folder) if is_image_file(f)])
#     assert len(files) > 0, f"No images in {folder}"
#     k = min(k, len(files))
#     idx = rng.choice(len(files), size=k, replace=False)
#     return [files[i] for i in idx]

# def read_gray(path):
#     img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
#     if img is None: raise ValueError(f"Cannot read: {path}")
#     return img

# def preprocess_resize(img_np, H, W):
#     """
#     Resize ảnh về (H,W), normalize [0,1], tensor (1,1,H,W)
#     """
#     rs = cv2.resize(img_np, (W, H), interpolation=cv2.INTER_AREA)
#     x  = torch.from_numpy(rs.astype(np.float32) / 255.0)[None, None, ...]
#     return x.to(DEVICE)

# def postprocess_mask(prob_map_np, orig_shape, thr=0.5):
#     """
#     prob_map_np: (Himg,Wimg) in [0,1] (cùng size với model)
#     Resize về kích thước gốc, threshold -> uint8 mask (0/255)
#     """
#     H0, W0 = orig_shape
#     p = cv2.resize(prob_map_np.astype(np.float32), (W0, H0), interpolation=cv2.INTER_LINEAR)
#     mask = (p >= thr).astype(np.uint8) * 255
#     return mask

# @torch.no_grad()
# def infer_mask(model, x):
#     """
#     model(x) -> (kl, xHat)
#     xHat shape: (B,1,H,W). Với bài toán segment, xHat là logits/prob mask
#     """
#     kl, xHat = model(x)
#     # Nếu giá trị không trong [0,1], coi là logits và áp sigmoid
#     if (xHat.min() < 0.0) or (xHat.max() > 1.0):
#         xHat = torch.sigmoid(xHat)
#     prob = xHat.squeeze(0).squeeze(0).detach().cpu().numpy()
#     return prob

# def predict_mask(img_path, thr=0.5, save=True, prefix=""):
#     img = read_gray(img_path)
#     H0, W0 = img.shape[:2]
#     x = preprocess_resize(img, Himg, Wimg)
#     prob = infer_mask(model, x)             # (Himg,Wimg) in [0,1]
#     mask = postprocess_mask(prob, (H0, W0), thr)
#     if save:
#         name = os.path.splitext(os.path.basename(img_path))[0]
#         outp = os.path.join(SAVE_DIR, f"{prefix}{name}_mask.png")
#         cv2.imwrite(outp, mask)
#     return img, mask

# def to3(x): 
#     return np.stack([x,x,x],axis=-1) if x.ndim==2 else x

# def make_pair(original, mask):
#     return np.hstack([to3(original), to3(mask)])

# def build_montage(paths, prefix=""):
#     rows = []
#     for p in paths:
#         img,mask = predict_mask(p, prefix=prefix)
#         rows.append(make_pair(img, mask))
#     # pad theo độ rộng lớn nhất (phòng khi size khác nhau)
#     maxw = max(r.shape[1] for r in rows)
#     rows2=[]
#     for r in rows:
#         h,w,c=r.shape
#         if w<maxw:
#             pad = np.zeros((h, maxw-w, c), dtype=r.dtype)
#             r = np.hstack([r,pad])
#         rows2.append(r)
#     return np.vstack(rows2)

# # ----------------------------
# # 5) Lấy mẫu & tạo montage
# # ----------------------------
# normal_paths = pick_samples(NORMAL_DIR, k=3, seed=2025)
# pneu_paths   = pick_samples(PNEU_DIR,   k=3, seed=2025)

# montage_normal = build_montage(normal_paths, prefix="NORMAL_")
# montage_pneu   = build_montage(pneu_paths,   prefix="PNEUMONIA_")

# # Lưu montage
# mn_path = os.path.join(SAVE_DIR,"montage_NORMAL_orig_mask.png")
# mp_path = os.path.join(SAVE_DIR,"montage_PNEUMONIA_orig_mask.png")
# cv2.imwrite(mn_path, montage_normal[:,:,::-1])
# cv2.imwrite(mp_path, montage_pneu[:,:,::-1])

# # ----------------------------
# # 6) Hiển thị
# # ----------------------------
# plt.figure(); plt.imshow(montage_normal); plt.axis('off'); plt.title("NORMAL: [Original | Mask] x3")
# plt.figure(); plt.imshow(montage_pneu);   plt.axis('off'); plt.title("PNEUMONIA: [Original | Mask] x3")

# print("[SAVED]")
# print(" -", mn_path)
# print(" -", mp_path)


In [2]:
# ===========================================
#  Precompute Segmentation Dataset (LungVAE)
#  - Generate: mask_soft.npy (model output size)
#  - Create: META/index.csv, META/segmenter.json
#  (Bản thu gọn - Chỉ lưu mask mềm)
# ===========================================
import os, json, csv, time, math, hashlib
import cv2
import numpy as np
from tqdm import tqdm
import torch
import torch.nn as nn
import torch.nn.functional as F

# ----------------------------
# 0) Cấu hình
# ----------------------------
DATA_ROOT = "/kaggle/input/chest-xray-pneumonia/chest_xray/chest_xray"
WEIGHTS_PATH = "/kaggle/input/lungvae/lungVAE.pt"
SAVE_ROOT = "./chest_xray_segmented_v1"   # output dataset

os.makedirs(SAVE_ROOT, exist_ok=True)
os.makedirs(os.path.join(SAVE_ROOT, "META"), exist_ok=True)

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print("[INFO] Device:", DEVICE)

# ----------------------------
# 1) Dán class uVAE (bạn đã cung cấp) - giữ nguyên
# ----------------------------
from torch.distributions import Normal, Independent
from torch.distributions.kl import kl_divergence as KLD
from torch.nn.functional import softplus, sigmoid, softmax

class convBlock(nn.Module):
    def __init__(self, inCh, nhid, nOp, pool=True,
                     ker=3,padding=1,pooling=2):
        super(convBlock,self).__init__()
        self.enc1 = nn.Conv2d(inCh,nhid,kernel_size=ker,padding=1)
        self.enc2 = nn.Conv2d(nhid,nOp,kernel_size=ker,padding=1)
        self.bn = nn.BatchNorm2d(inCh)
        if pool:
            self.scale = nn.AvgPool2d(kernel_size=pooling)
        else:
            self.scale = nn.Upsample(scale_factor=pooling)
        self.pool = pool
        self.act = nn.ReLU()
    def forward(self,x):
        x = self.scale(x)
        x = self.bn(x)
        x = self.act(self.enc1(x))
        x = self.act(self.enc2(x))
        return x

class uVAE(nn.Module):
    def __init__(self, nlatent,unet=False, 
                     nhid=8, ker=3, inCh=1,h=640,w=512):
        super(uVAE, self).__init__()
        self.latent_space = nlatent
        self.unet = unet
        if not self.unet:
            self.enc11 = nn.Conv2d(inCh,nhid,kernel_size=ker,padding=1)
            self.enc12 = nn.Conv2d(nhid,nhid,kernel_size=ker,padding=1)
            self.enc2 = convBlock(nhid,2*nhid,2*nhid,pool=True)
            self.enc3 = convBlock(2*nhid,4*nhid,4*nhid,pool=True)
            self.enc4 = convBlock(4*nhid,8*nhid,8*nhid,pool=True)
            self.enc5 = convBlock(8*nhid,16*nhid,16*nhid,pool=True)
            self.bot11 = nn.Conv1d(16*nhid,1,kernel_size=1)
            self.bot12 = nn.Conv1d(int((h/16)*(w/16)),2*nlatent,kernel_size=1)
            self.bot21 = nn.Conv1d(nlatent,int((h/64)*(w/64)),kernel_size=1)
            self.bot22 = nn.Conv1d(1,nhid,kernel_size=1)
            self.bot23 = nn.Conv1d(nhid,4*nhid,kernel_size=1)
            self.bot24 = nn.Conv1d(4*nhid,16*nhid,kernel_size=1)
        self.uEnc11 = nn.Conv2d(inCh,nhid,kernel_size=ker,padding=1)
        self.uEnc12 = nn.Conv2d(nhid,nhid,kernel_size=ker,padding=1)
        self.uEnc2 = convBlock(nhid,2*nhid,2*nhid,pool=True,pooling=4)
        self.uEnc3 = convBlock(2*nhid,4*nhid,4*nhid,pool=True,pooling=4)
        self.uEnc4 = convBlock(4*nhid,8*nhid,8*nhid,pool=True)
        self.uEnc5 = convBlock(8*nhid,16*nhid,16*nhid,pool=True)
        if not self.unet:
            self.dec5 = convBlock(32*nhid,8*nhid,8*nhid,pool=False)
        else:
            self.dec5 = convBlock(16*nhid,8*nhid,8*nhid,pool=False)
        self.dec4 = convBlock(16*nhid,4*nhid,4*nhid,pool=False)
        self.dec3 = convBlock(8*nhid,2*nhid,2*nhid,pool=False,pooling=4)
        self.dec2 = convBlock(4*nhid,nhid,nhid,pool=False,pooling=4)
        self.dec11 = nn.Conv2d(2*nhid,nhid,kernel_size=ker,padding=1)
        self.dec12 = nn.Conv2d(nhid,inCh,kernel_size=ker,padding=1)
        self.act = nn.ReLU()
        self.mu_0 = torch.zeros((1,nlatent))
        self.sigma_0 = torch.ones((1,nlatent))
        self.h = h
        self.w = w
    def vae_encoder(self,x):
        x = self.act(self.enc11(x)); x = self.act(self.enc12(x))
        x = self.enc2(x); x = self.enc3(x); x = self.enc4(x); x = self.enc5(x)
        z = self.act(self.bot11(x.view(x.shape[0],x.shape[1],-1)))
        z = self.bot12(z.permute(0,2,1)); return z.squeeze(-1)
    def unet_encoder(self,x_in):
        x = []; x.append(self.act(self.uEnc12(self.act(self.uEnc11(x_in)))))
        x.append(self.uEnc2(x[-1])); x.append(self.uEnc3(x[-1]))
        x.append(self.uEnc4(x[-1])); x.append(self.uEnc5(x[-1]))
        return x
    def decoder(self,x_enc,z=None):
        if not self.unet:
            x = self.act(self.bot21(z.unsqueeze(2))); x = self.act(self.bot22(x.permute(0,2,1)))
            x = self.act(self.bot23(x)); x = self.act(self.bot24(x))
            x = x.view(x.shape[0],x.shape[1],int(self.h/64),int(self.w/64))
            x = torch.cat((x,x_enc[-1]),dim=1); x = self.dec5(x)
        else:
            x = self.dec5(x_enc[-1])
        x = torch.cat((x,x_enc[-2]),dim=1); x = self.dec4(x)
        x = torch.cat((x,x_enc[-3]),dim=1); x = self.dec3(x)
        x = torch.cat((x,x_enc[-4]),dim=1); x = self.dec2(x)
        x = torch.cat((x,x_enc[-5]),dim=1); x = self.act(self.dec11(x)); x = self.dec12(x)
        return x
    def forward(self, x):
        kl = torch.zeros(1, device=x.device); z = 0.
        x_enc = self.unet_encoder(x)
        if not self.unet:
            emb = self.vae_encoder(x); mu, log_var = torch.chunk(emb, 2, dim=1)
            log_var = softplus(log_var); sigma = torch.exp(log_var / 2)
            posterior = Independent(Normal(loc=mu,scale=sigma),1)
            z = posterior.rsample()
            prior = Independent(Normal(loc=self.mu_0.to(x.device),scale=self.sigma_0.to(x.device)),1)
            kl = KLD(posterior,prior).sum()
        xHat = self.decoder(x_enc,z); return kl, xHat

# ----------------------------
# 2) Utils (Đã thu gọn)
# ----------------------------
def is_image_file(p): 
    return p.lower().endswith((".png",".jpg",".jpeg",".bmp",".tif",".tiff"))

def sha256_of_file(path, chunk=1024*1024):
    h = hashlib.sha256()
    with open(path, "rb") as f:
        while True:
            b = f.read(chunk)
            if not b: break
            h.update(b)
    return h.hexdigest()

def ensure_dirs(base, split, label):
    # Chỉ tạo thư mục mask_soft
    base2 = os.path.join(base, split, label, "mask_soft")
    os.makedirs(base2, exist_ok=True)
    return base2

def read_gray(path):
    img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
    if img is None:
        raise ValueError(f"Cannot read: {path}")
    return img

def preprocess_resize(img_np, H, W, device):
    rs = cv2.resize(img_np, (W, H), interpolation=cv2.INTER_AREA)
    x  = torch.from_numpy(rs.astype(np.float32)/255.0)[None, None, ...]  # (1,1,H,W)
    return x.to(device)

# ----------------------------
# 3) Load weights & infer hparams
# ----------------------------
def infer_hparams_from_state_dict(sd):
    nhid = sd["uEnc11.weight"].shape[0] if "uEnc11.weight" in sd else 8
    has_vae = ("bot21.weight" in sd) or ("bot12.weight" in sd)
    unet_flag = not has_vae
    nlatent = 16
    if "bot21.weight" in sd:
        nlatent = sd["bot21.weight"].shape[1]
    h, w = 640, 512
    if "bot12.weight" in sd:
        in_c = sd["bot12.weight"].shape[1]; candidates = []
        for H in [512, 576, 600, 640, 672, 704, 720, 736, 768, 800, 832, 896, 960, 1024]:
            for W in [384, 400, 448, 480, 512, 544, 576, 600, 640, 672, 704, 720, 736, 768, 800, 896, 960, 1024]:
                if (H % 16 == 0) and (W % 16 == 0) and (H//16)*(W//16) == in_c:
                    candidates.append((H,W))
        if (640,512) in candidates: h,w = 640,512
        elif candidates: h,w = candidates[0]
    return nlatent, nhid, h, w, unet_flag

obj = torch.load(WEIGHTS_PATH, map_location="cpu")
sd = obj["state_dict"] if isinstance(obj, dict) and "state_dict" in obj else obj
assert isinstance(sd, dict), "Weights phải là state_dict hoặc checkpoint chứa state_dict."

nlatent, nhid, Himg, Wimg, unet_flag = infer_hparams_from_state_dict(sd)
print(f"[INFO] Inferred: nlatent={nlatent}, nhid={nhid}, H×W={Himg}×{Wimg}, unet={unet_flag}")

model = uVAE(nlatent=nlatent, unet=unet_flag, nhid=nhid, inCh=1, h=Himg, w=Wimg)
missing, unexpected = model.load_state_dict(sd, strict=False)
print("[INFO] load_state_dict missing:", len(missing), "unexpected:", len(unexpected))
model.to(DEVICE).eval()

# ----------------------------
# 4) Chuẩn bị META (Đã thu gọn)
# ----------------------------
weights_sha = sha256_of_file(WEIGHTS_PATH)
meta = {
    "created_at": time.strftime("%Y-%m-%d %H:%M:%S"),
    "seg_model": "uVAE (LungVAE)",
    "weights_path": WEIGHTS_PATH,
    "weights_sha256": weights_sha,
    "params": {
        "nlatent": int(nlatent),
        "nhid": int(nhid),
        "input_size": [int(Himg), int(Wimg)],
        "unet_mode": bool(unet_flag),
    },
    "pipeline": {
        "resize_in": f"{Himg}x{Wimg}",
        "mask_type": f"soft (prob) saved as .npy ({Himg}x{Wimg})",
    }
}
with open(os.path.join(SAVE_ROOT, "META", "segmenter.json"), "w") as f:
    json.dump(meta, f, indent=2)

csv_path = os.path.join(SAVE_ROOT, "META", "index.csv")
# Thu gọn cột CSV
csv_cols = ["split","label","orig_path","mask_soft_path", "H","W",
            "seg_model","weights_sha256", "resize_in"]
csv_f = open(csv_path, "w", newline="")
csv_w = csv.DictWriter(csv_f, fieldnames=csv_cols)
csv_w.writeheader()

# ----------------------------
# 5) Loop qua train/val/test & NORMAL/PNEUMONIA (Đã thu gọn)
# ----------------------------
splits = [d for d in ["train","val","test"] if os.path.isdir(os.path.join(DATA_ROOT,d))]
labels = ["NORMAL","PNEUMONIA"]

@torch.no_grad()
def predict_soft_mask(img_np):
    x = preprocess_resize(img_np, Himg, Wimg, DEVICE)
    kl, xHat = model(x)
    if (xHat.min() < 0.0) or (xHat.max() > 1.0):
        xHat = torch.sigmoid(xHat)
    prob = xHat.squeeze(0).squeeze(0).detach().cpu().numpy()  # (Himg,Wimg)
    return prob

for split in splits:
    for label in labels:
        in_dir = os.path.join(DATA_ROOT, split, label)
        if not os.path.isdir(in_dir): 
            continue

        # base2 bây giờ trỏ thẳng đến thư mục .../mask_soft/
        base2_mask_soft = ensure_dirs(SAVE_ROOT, split, label) 
        files = sorted([f for f in os.listdir(in_dir) if is_image_file(f)])
        print(f"[INFO] {split}/{label}: {len(files)} files")

        for fname in tqdm(files):
            src_path = os.path.join(in_dir, fname)
            img = read_gray(src_path)
            H0, W0 = img.shape[:2]

            # 1. Dự đoán soft mask (model size)
            prob_small = predict_soft_mask(img)

            # ----- Chỉ lưu file mask mềm -----
            name = os.path.splitext(fname)[0]
            
            # 2. Lưu mask mềm
            out_mask_soft = os.path.join(base2_mask_soft, f"{name}_mask_soft.npy")
            np.save(out_mask_soft, prob_small.astype(np.float16))

            # 3. Ghi CSV (relative paths)
            row = {
                "split": split,
                "label": 0 if label=="NORMAL" else 1,
                "orig_path": src_path,
                "mask_soft_path": os.path.relpath(out_mask_soft, SAVE_ROOT),
                "H": H0, "W": W0,
                "seg_model": "uVAE (LungVAE)",
                "weights_sha256": weights_sha,
                "resize_in": f"{Himg}x{Wimg}",
            }
            csv_w.writerow(row)

csv_f.close()
print("[DONE] Wrote dataset to:", SAVE_ROOT)
print(" - META/segmenter.json")
print(" - META/index.csv")

[INFO] Device: cuda
[INFO] Inferred: nlatent=8, nhid=16, H×W=640×512, unet=False
[INFO] load_state_dict missing: 0 unexpected: 0
[INFO] train/NORMAL: 1341 files


100%|██████████| 1341/1341 [01:13<00:00, 18.25it/s]


[INFO] train/PNEUMONIA: 3875 files


100%|██████████| 3875/3875 [02:27<00:00, 26.28it/s]


[INFO] val/NORMAL: 8 files


...

[INFO] test/NORMAL: 234 files


100%|██████████| 234/234 [00:10<00:00, 21.63it/s]


[INFO] test/PNEUMONIA: 390 files


100%|██████████| 390/390 [00:13<00:00, 27.91it/s]

[DONE] Wrote dataset to: ./chest_xray_segmented_v1
 - META/segmenter.json
 - META/index.csv
